In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[5]))

,Demand,Normalized_net,Temprature
0,2338.0,0.443389,-5.8
1,2164.1,0.410410,-6.1
2,2060.0,0.390668,-7.5
3,1989.3,0.377260,-8.4
4,1968.8,0.373372,-7.4
...,...,...,...
43819,2574.2,0.488183,3.4
43820,2446.3,0.463928,3.3
43821,2324.1,0.440753,2.6
43822,2201.8,0.417560,2.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[5]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_LongIsland,Temp_LongIsland
0,2338.0,-5.8
1,2164.1,-6.1
2,2060.0,-7.5
3,1989.3,-8.4
4,1968.8,-7.4
...,...,...
43819,2574.2,3.4
43820,2446.3,3.3
43821,2324.1,2.6
43822,2201.8,2.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_LongIsland,Temp_LongIsland,Lag-2,Lag-7
0,2462.3,-13.9,2173.5,2338.0
1,2366.1,-14.2,2080.4,2164.1
2,2310.6,-14.8,2034.1,2060.0
3,2288.1,-15.6,2017.9,1989.3
4,2320.7,-16.0,2045.2,1968.8
...,...,...,...,...
43651,2574.2,3.4,2514.0,2508.5
43652,2446.3,3.3,2446.4,2438.5
43653,2324.1,2.6,2339.4,2376.6
43654,2201.8,2.7,2183.3,2292.6


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 18s 12ms/step - loss: 0.0101 - mape: 6144.2979 - mae: 0.0757 - val_loss: 0.0054 - val_mape: 90.2473 - val_mae: 0.0583
Epoch 2/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0059 - mape: 5385.3618 - mae: 0.0579 - val_loss: 0.0065 - val_mape: 91.0018 - val_mae: 0.0580
Epoch 3/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0041 - mape: 5047.0391 - mae: 0.0483 - val_loss: 0.0098 - val_mape: 109.6501 - val_mae: 0.0750
Epoch 4/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0037 - mape: 4974.5723 - mae: 0.0454 - val_loss: 0.0071 - val_mape: 80.5082 - val_mae: 0.0600
Epoch 5/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0033 - mape: 4692.1582 - mae: 0.0432 - val_loss: 0.0073 - val_mape: 63.6279 - val_mae: 0.0558
Epoch 6/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0031 - mape: 4570.0137 - mae: 0.0417 - val_

1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 1329.4363 - mae: 0.0304 - val_loss: 0.0061 - val_mape: 62.6211 - val_mae: 0.0510
Epoch 48/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 1228.1371 - mae: 0.0303 - val_loss: 0.0061 - val_mape: 66.2661 - val_mae: 0.0488
Epoch 49/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 1252.8407 - mae: 0.0302 - val_loss: 0.0063 - val_mape: 60.3358 - val_mae: 0.0507
Epoch 50/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 1444.7839 - mae: 0.0300 - val_loss: 0.0063 - val_mape: 63.7685 - val_mae: 0.0500
Epoch 51/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 1330.1405 - mae: 0.0299 - val_loss: 0.0061 - val_mape: 65.7152 - val_mae: 0.0490
Epoch 52/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0016 - mape: 1313.9755 - mae: 0.0295 - val_loss: 0.

1105/1105 [==============================] - 12s 11ms/step - loss: 9.6994e-04 - mape: 1072.5978 - mae: 0.0231 - val_loss: 0.0077 - val_mape: 73.8594 - val_mae: 0.0608
Epoch 94/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.6029e-04 - mape: 1209.5038 - mae: 0.0230 - val_loss: 0.0074 - val_mape: 83.9174 - val_mae: 0.0599
Epoch 95/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.3443e-04 - mape: 1388.8435 - mae: 0.0227 - val_loss: 0.0072 - val_mape: 73.1502 - val_mae: 0.0585
Epoch 96/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.4186e-04 - mape: 1237.7126 - mae: 0.0227 - val_loss: 0.0072 - val_mape: 87.2414 - val_mae: 0.0610
Epoch 97/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0010 - mape: 1321.5331 - mae: 0.0233 - val_loss: 0.0076 - val_mape: 85.5059 - val_mae: 0.0626
Epoch 98/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.0981e-04 - mape: 996.0815 - mae: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1240.921546459198


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.21827254, 0.20344839, 0.19795945, ..., 0.3420808 , 0.30420035,
       0.26950797], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.40230817, 0.37525055, 0.36036315, ..., 0.33106785, 0.30314256,
       0.27766044])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,2636.1,1830.108276
1,2517.6,1765.185181
2,2452.4,1741.146240
3,2428.5,1736.122681
4,2459.8,1793.647583
...,...,...
4363,2574.2,2570.896973
4364,2446.3,2502.732666
4365,2324.1,2372.331787
4366,2201.8,2206.432617


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.981711143565349